In [1]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import velocity_estimation.two_dim_velocity_estimates as tdve
import velocity_estimation.utils as u
import velocity_estimation.time_delay_estimation as td
from fppanalysis.running_moments import window_radius, run_norm_ds, run_mean_ds
import cosmoplots
import tde_functions.plot_velocity_estimation as plot_velocity_estimation

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


ModuleNotFoundError: No module named 'tde_functions'

#### Open dataset

In [ ]:
shot = 1111208034
ds = xr.open_dataset("Path to your apd data")

#### Remove dead pixels

In [ ]:
path_dead_pixel = Path('Path to dead pixels')
dead_pix = np.load(
    str(path_dead_pixel)
    + "/"
    + f"dead_pixels_shot_{shot}.npz"
)
dead_pix_arr = dead_pix['dead_pix_arr'] # Don't flip array, quiver plot is oriented this way
dead_pix_arr_ds = dead_pix_arr[::-1,:] # Flip array to match ds['frames'] orientation
for y in range(dead_pix_arr_ds.shape[0]):
    for x in range(dead_pix_arr_ds.shape[1]):
        if dead_pix_arr_ds[9-y, x]:
            ds['frames'].loc[dict(y=y, x=x)] = np.nan

#### For a given start and end time, slice the dataset

In [ ]:
t_start = 0
t_end = 0
sliced_ds = ds.sel(time=slice(t_start, t_end))

#### Detrend data

In [ ]:
cut_off_freq = 1e3
radius = window_radius(cut_off_freq, sliced_ds.time)
ds = run_norm_ds(sliced_ds, radius)
ds = run_mean_ds(ds, 3)

#### Estimation options for velocity estimation

In [ ]:
eo = tdve.EstimationOptions()
eo.use_3point_method = True
eo.cc_options.running_mean = True
eo.cc_options.minimum_cc_value = 0
eo.neighbour_options = tdve.NeighbourOptions(ccf_min_lag=0, max_separation=1, min_separation=1)

#### Generate velocities

In [ ]:
movie_data = tdve.estimate_velocity_field(u.CModImagingDataInterface(ds), eo)
vx = movie_data.get_vx()
vy = movie_data.get_vy()
confidences = movie_data.get_confidences()
R = movie_data.get_R()
Z = movie_data.get_Z()

#### Plot velocity field

In [ ]:
plot_velocity_estimation(
    shot, # At this point, shot is only used for the title
    t_start, 
    t_end, 
    movie_data, 
    dead_pix_arr, 
    LCFS_limiter_foldername,
    title=None,
)